
# Advanced Data Science Capstone

## Correlation of air pollution and Prevalence of Asthma bronchiale in Germany  

## Feature Creation and Feature engineering

### The deliverables
The deliverables of the current stage:

 - Spark DataFrames with disease prevalence column, county id, and some features extracted from air pollution data series for sensors located in corresponding county

###  Feature creation
The basic features for air pollution levels are

 - Number of hours when pollutant concentration exceeded some certain value
 - Mean or Median concentration of the pollutant
 
###  Feature quality check

 - Feature variance
 - Feature cross-correlation matrix
 
The necessary libraries and the data sets preprocessed at the ETL stage loaded:

In [1]:
##### Libraries:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re

Waiting for a Spark session to start...
Spark Initialization Done! ApplicationId = app-20190822081237-0001
KERNEL_ID = 79ad53a1-2597-4ba6-b2df-4485bc1acd78


In [2]:
# The code was removed by Watson Studio for sharing.

###  Feature creation

Now let's create some basic features, illustrating some integral quantities of air pollution over the year.
For the start the following features will be generated:
 - Average concentration of every kind of pollutant over the year (average over all sensors within the county)
 - 50th and 75th percentile of every kind pollutant over the year, that is also proportional to the number of days when pollutant concentration exceeded some certain value

In [3]:
#dfAllSpark.createOrReplaceTempView("SensorsHour")
#SensorLocationSpark.createOrReplaceTempView("SensorsCounty")

#sql view generation:
dfAllLongSpark.createOrReplaceTempView("SensorsHour")
dfAsthmaSpark.createOrReplaceTempView("DiseaseCounty")

spark.sql("select * from SensorsHour limit 10").show()

spark.sql("select * from DiseaseCounty limit 10").show()          

+-------------+---------+--------+
|PollutantConc|Pollutant|CountyID|
+-------------+---------+--------+
|      15.5531|      NO2|   15002|
|      15.5153|      NO2|   15002|
|      19.5611|      NO2|   15002|
|      20.2804|      NO2|   15002|
|     28.63875|      NO2|   15002|
|      50.6439|      NO2|   15002|
|     62.99845|      NO2|   15002|
|       55.207|      NO2|   15002|
|      59.3478|      NO2|   15002|
|      56.0626|      NO2|   15002|
+-------------+---------+--------+

+--------+--------+
|CountyID|DiseaseR|
+--------+--------+
|   16056|     8.9|
|   16072|     8.7|
|    3451|     8.5|
|   16073|     8.3|
|    3151|     8.2|
|    3403|     8.2|
|    1059|     8.1|
|    3156|     8.1|
|    8128|     8.0|
|    5570|     7.9|
+--------+--------+



In [ ]:
spark.sql("drop TABLE PollutantPercentiles")

In [ ]:
     
#--  (select percentile(PollutantConc,  0.5) over(PARTITION BY Pollutant, CountyID) from SensorsHour) as Percentile50
#
#--     group by Pollutant, CountyID)
#--,
#--      (select percentile(PollutantConc, 0.75) as Percentile75 from SensorsHour group by Pollutant, CountyID),


#agent_id|payment_amount|
#+--------+--------------+
#|       a|          1000|
#|       b|          1100|
#
# select agent_id, percentile_approx(payment_amount,0.95) as approxQuantile from df group by agent_id


In [6]:
dfAllLongSpark.printSchema()
dfAsthmaSpark.printSchema()

root
 |-- PollutantConc: double (nullable = true)
 |-- Pollutant: string (nullable = true)
 |-- CountyID: long (nullable = true)

root
 |-- CountyID: long (nullable = true)
 |-- DiseaseR: double (nullable = true)



In [7]:
from pyspark.sql.types import *
#StructType, StructField, IntegerType, StringType

schema = StructType([
    StructField("Pollutant", StringType(), True), 
    StructField("CountyID", LongType(), True),
    StructField("Mean", DoubleType(), True),
    StructField("Percentile50", DoubleType(), True),
    StructField("Percentile75", DoubleType(), True)    
   ])

# or df = sc.parallelize([]).toDF(schema)
# Spark < 2.0 
# sqlContext.createDataFrame([], schema)

dfPollutantPercentilesSpark = spark.createDataFrame([], schema)


#dfPollutantPercentilesSpark = sqlContext.createDataFrame(sc.emptyRDD())
dfPollutantPercentilesSpark.createOrReplaceTempView("PollutantPercentiles")
dfPollutantPercentilesSpark.printSchema()

root
 |-- Pollutant: string (nullable = true)
 |-- CountyID: long (nullable = true)
 |-- Mean: double (nullable = true)
 |-- Percentile50: double (nullable = true)
 |-- Percentile75: double (nullable = true)



In [11]:
spark.sql("select * from PollutantPercentiles limit 10").show()

+---------+--------+----+------------+------------+
|Pollutant|CountyID|Mean|Percentile50|Percentile75|
+---------+--------+----+------------+------------+
+---------+--------+----+------------+------------+



In [14]:
#UPDATE table1 
#SET a = t2.a, b = t2.b, .......
#FROM table2 t2
#WHERE table1.id = t2.id


#spark.sql("INSERT INTO TABLE appendTest SELECT id, name, age, null, null FROM inputTable");

#spark.sql("""INSERT INTO TABLE PollutantPercentiles 

spark.sql("""
SELECT distinct 
     Pollutant, CountyID,
     AVG(PollutantConc) over(PARTITION BY Pollutant, CountyID) AS Mean,
     percentile(PollutantConc,  0.5) over(PARTITION BY Pollutant, CountyID) as Percentile50,
     percentile(PollutantConc, 0.75) over(PARTITION BY Pollutant, CountyID) as Percentile75
FROM SensorsHour
""").show(10)          

#""") 


KeyboardInterrupt: 

In [ ]:
spark.sql("""CREATE TABLE PollutantPercentiles AS(
SELECT distinct 
     Pollutant, CountyID,
     AVG(PollutantConc) over(PARTITION BY Pollutant, CountyID) AS Mean,
     percentile(PollutantConc,  0.5) over(PARTITION BY Pollutant, CountyID) as Percentile50,
     percentile(PollutantConc, 0.75) over(PARTITION BY Pollutant, CountyID) as Percentile75
FROM SensorsHour)
""")

In [ ]:
spark.sql("select * from PollutantPercentiles limit 10").show()

In [ ]:
spark.sql("drop TABLE PolMeanLongDisease50perc")

In [ ]:
# PolMeanLongDisease50perc : Pollutant mean; Disease level at or above 50th percentile
# pollutant list: ListOfPollutantsLong = ['NO','NO2','PM1']

spark.sql("""
CREATE TABLE PolMeanLongDisease50perc AS 
      (
       SELECT 
         CountyID,
         (case 
            WHEN DiseaseR >=
             (select percentile(DiseaseR, 0.75) from DiseaseCounty limit 1) 
--           (select top 1 percentile_cont(.75) within group (order by DiseaseR) over() from DiseaseCounty) 
          THEN 1
          ELSE 0
         END 
        ) as DiseaseRFeat
     FROM DiseaseCounty)
""")
              

          
# Feature matrices to be created:
#dfPolMeanLongDisease50perc = DiseaseFeaturePercentile(FeatureSetLongMean, 50.0)
#dfPolMeanLongDisease75perc = DiseaseFeaturePercentile(FeatureSetLongMean, 75.0)
#dfPolMeanLongDisease95perc = DiseaseFeaturePercentile(FeatureSetLongMean, 95.0)
#
#dfPolLongPerc75Disease50perc = DiseaseFeaturePercentile(FeatureSetLongPerc75, 50.0)
#dfPolLongPerc75Disease75perc = DiseaseFeaturePercentile(FeatureSetLongPerc75, 75.0)
#dfPolLongPerc75Disease95perc = DiseaseFeaturePercentile(FeatureSetLongPerc75, 95.0)
          
# ListOfPollutantsDense = ['NO','NO2','NOx','O3','PM1']    

In [ ]:
spark.sql("select * from PolMeanLongDisease50perc limit 10").show()

In [ ]:
spark.sql("""
ALTER TABLE PolMeanLongDisease50perc ADD COLUMNS (NO double, NO2 double, PM1 double)
""").show()


In [ ]:
spark.sql("""
UPDATE PolMeanLongDisease50perc
SET PolMeanLongDisease50perc.NO=(SELECT PollutantPercentiles.Mean
  FROM PollutantRercentiles
  WHERE PollutantRercentiles.Pollutant='NO' AND PollutantRercentiles.CountyID=PolMeanLongDisease50perc.CountyID);
""").show()

#### Mean pollutant concentration over the year
The quantity can be straightforward extracted using *.describe()* method of the data frame:

In [ ]:
dffAllSummary=dffAll.describe()

def MeanByPollutantCountyID(Pollutant, CountyID):
    PollutantMasc='^.{8}' + Pollutant + '_'
# collecting set of all sensors in the given county:    
    CountyStationSet = SensorLocation.loc[SensorLocation['countyid']==CountyID]['SensorID'].tolist()
    if (CountyStationSet == []) or ([col for col in dffAllSummary.columns if re.search('(^.{7})',col).group(1) in CountyStationSet if ('_'+Pollutant+'_') in col] == []):
        return(np.nan)
    try:
        dffAllPollutantCountyID = dffAllSummary[[col for col in dffAllSummary.columns if re.search('(^.{7})',col).group(1) in CountyStationSet]].filter(regex=PollutantMasc,axis=1)
        return(dffAllPollutantCountyID.loc[['mean']].mean(axis=1)[0])
    except:    
        return(np.nan)

In [ ]:
ListOfPollutants = ['CO','NO','NO2','NOx','O3','PM1','PM2', 'PM10', 'SO2', 'CHB', 'CH4', 'C8H10', 'CHT', 'CO2']

for pollutant in ListOfPollutants:
    ColIndex=pollutant+'mean'
    dfDisease[ColIndex] =dfDisease.apply(lambda x: MeanByPollutantCountyID(pollutant, x['Regions-ID']), axis=1)

In [ ]:
# rewriting two functions above with SQL:
# need: dfDiseasePolMean[pollutant] = df with columns: CountyID, pollutantMeanOverSensorsInThisCounty




In [ ]:
#dfDisease.head(53)
dfDisease.isna().sum()

From the table above one can see, that only 5 pollutants (*NO, NO2, NOx, O3, PM1*) were measured in about a half of German counties (total number of counties mentioned in the *heart failures* and *asthma bronchiale* datasets is 402). Let's take a look, in how many counties all 6 pollutants were measured simultaneously:


In [ ]:
FeatureSetDenseMean = dfDisease[['Regions-ID','Wert','NOmean','NO2mean','NOxmean','O3mean','PM1mean']].dropna().reset_index(drop=True)
FeatureSetDenseMean.columns=['CountyID','DiseaseR','NOmean','NO2mean','NOxmean','O3mean','PM1mean']

In [ ]:
#FeatureSetDenseMean.tail(55)
FeatureSetDenseMean.describe()
#FeatureSetDenseMean.isna().sum()
#FeatureSetDenseMean['NOmean'].mean()

From the table above one can conclude, that despite only about a third of German counties are included into the **FeatureSetDense** dataset, all the measured quantities have good variability. In order to illustrate it, the histograms are plotted:

In [ ]:
FeatureSetDenseMean.hist(column=['NOmean','NO2mean','NOxmean','O3mean','PM1mean']);

We can also maximize number of counties in the dataset, removing 2 of 5 pollutants. It is found, that for the current dataset it will lead to increase of observations up to 199, almost a half of the counties:

In [ ]:
FeatureSetLongMean = dfDisease[['Regions-ID','Wert','NOmean','NO2mean','PM1mean']].dropna().reset_index(drop=True)
FeatureSetLongMean.columns=['CountyID','DiseaseR','NOmean','NO2mean','PM1mean']
FeatureSetLongMean.describe()

Writing feature files:

In [ ]:
!rm -rf Capstone.FeatureEng
!mkdir Capstone.FeatureEng
FeatureSetDenseMean.to_csv('Capstone.FeatureEng/Capstone.feature_eng.DenseMean.1.0.csv', index=False)
FeatureSetLongMean.to_csv('Capstone.FeatureEng/Capstone.feature_eng.LongMean.1.0.csv', index=False)

#### Quantile-based features
Constructing the global distribution of selected pollutant concentration over all counties of interest,
one can find e.g. 75th quantile of this pollutant concentration distribution, 
and than count number of hours, when this concentration was exceeded in each county.
This quantity is some generalization of the *"number of days when the pollutant limit has been exceeded"* quantity, because it tune itself to pollution levels available in the dataset.

In [ ]:
# Function for Croppind of initial dffAll dataset to columns reflecting measurements 
# of pollutants in ListOfPollutants at counties in ListOfCountyIDs:
def CropPollutantsIDsTS(ListOfPollutants, ListOfCountyIDs):
    CountyStationSet=[]
    dffAllCrop=pd.DataFrame(dffAll['observation_period'])

    for CountyID in ListOfCountyIDs:
        CountyStationSet.extend(SensorLocation.loc[SensorLocation['countyid']==CountyID]['SensorID'].tolist())
    for Pollutant in ListOfPollutants:
        PollutantMasc='^.{8}' + Pollutant + '_' 
        dffAllCrop=dffAllCrop.join(dffAll[[col for col in dffAll.columns if re.search('(^.{7})',col).group(1) in CountyStationSet]].filter(regex=PollutantMasc,axis=1))
    return(dffAllCrop)

# Function for calculating Nth percentiles for pollutants in ListOfPollutants at counties in ListOfCountyIDs:
def ReturnNthPercentilePollutantsIDsTS(Percentile, ListOfPollutants, ListOfCountyIDs):
    CountyStationSet=[]
    NthPercentile=pd.DataFrame()

    for CountyID in ListOfCountyIDs:
        CountyStationSet.extend(SensorLocation.loc[SensorLocation['countyid']==CountyID]['SensorID'].tolist())
    for Pollutant in ListOfPollutants:
        PollutantMasc='^.{8}' + Pollutant + '_'
        NthPercentile[Pollutant] = [(dffAll[[col for col in dffAll.columns if re.search('(^.{7})',col).group(1) in CountyStationSet]].filter(regex=PollutantMasc,axis=1)).stack().reset_index(drop=True).quantile(Percentile/100.0)]
    return(NthPercentile)

# Function for calculating Feature matrix, containing average (over sensors) number of hours for each county, when pollutant concentration
# was exceeding Nth global percentiles for the pollutant at counties of interest (in ListOfCountyIDs):
def FeatureMatrixNthPercentile(Percentile, ListOfPollutants, ListOfCountyIDs):
    NthPercentile=ReturnNthPercentilePollutantsIDsTS(Percentile, ListOfPollutants, ListOfCountyIDs)
    dffAllCrop=CropPollutantsIDsTS(ListOfPollutants, ListOfCountyIDs)

    FeatureMatrix=pd.DataFrame({'CountyID': dfDisease.loc[dfDisease['Regions-ID'].isin(ListOfCountyIDs)]['Regions-ID'], 'DiseaseR': dfDisease.loc[dfDisease['Regions-ID'].isin(ListOfCountyIDs)]['Wert']}).reset_index(drop=True)
    for Pollutant in ListOfPollutants:
        PollutantHoursExceed=[]
        PollutantMasc='^.{8}' + Pollutant + '_'
        for CountyID in ListOfCountyIDs:
            CountyStationSet=[]
            CountyStationSet.extend(SensorLocation.loc[SensorLocation['countyid']==CountyID]['SensorID'].tolist())

            dffPollutantCounty = dffAllCrop[[col for col in dffAllCrop.columns if re.search('(^.{7})',col).group(1) in CountyStationSet]].filter(regex=PollutantMasc,axis=1)
            NumPollutantStationsAtCounty = dffPollutantCounty.shape[1]
            PollutantHoursExceed.append((dffPollutantCounty.stack().reset_index(drop=True)>NthPercentile[Pollutant][0]).reset_index(drop=True).sum()/NumPollutantStationsAtCounty)
        FeatureMatrix = FeatureMatrix.reset_index(drop=True).join(pd.DataFrame({Pollutant : PollutantHoursExceed}))        
    return(FeatureMatrix)

The same way as it was done for the pollutant mean as a feature, the quantile-based features with two types of pollutant lists (*Dense* and *Long*) and two percentile values (50 and 75) will be created:

In [ ]:
ListOfPollutantsDense = ['NO','NO2','NOx','O3','PM1']
ListOfCountyIDsDense = FeatureSetDenseMean['CountyID']
FeatureSetDensePerc50 = FeatureMatrixNthPercentile(50, ListOfPollutantsDense, ListOfCountyIDsDense)
FeatureSetDensePerc75 = FeatureMatrixNthPercentile(75, ListOfPollutantsDense, ListOfCountyIDsDense)

ListOfPollutantsLong = ['NO','NO2','PM1']
ListOfCountyIDsLong = FeatureSetLongMean['CountyID']
FeatureSetLongPerc50 = FeatureMatrixNthPercentile(50, ListOfPollutantsLong, ListOfCountyIDsLong)
FeatureSetLongPerc75 = FeatureMatrixNthPercentile(75, ListOfPollutantsLong, ListOfCountyIDsLong)

#FeatureSetDensePerc50.to_csv('Capstone.FeatureEng/Capstone.feature_eng.DensePerc50.1.0.csv', index=False)
#FeatureSetDensePerc75.to_csv('Capstone.FeatureEng/Capstone.feature_eng.DensePerc75.1.0.csv', index=False)

#FeatureSetLongPerc50.to_csv('Capstone.FeatureEng/Capstone.feature_eng.LongPerc50.1.0.csv', index=False)
#FeatureSetLongPerc75.to_csv('Capstone.FeatureEng/Capstone.feature_eng.LongPerc75.1.0.csv', index=False)

In [ ]:
#FeatureSetDensePerc50.tail(10)
FeatureSetDensePerc50.head(10)
#FeatureSetLongPerc50.tail(10)
#FeatureSetDensePerc75.tail(10)
#FeatureSetLongPerc75.tail(10)


###  Feature transformation

In this part the following notation is used:
 - *X, y* - features and labels of initial feature matrix.
 - *X_train0, X_test0, y_train0, y_test0* initial feature matrix, splitted into train and test sets
 - *X_train, X_test, y_train, y_test* normalized feature matrix, splitted into train and test sets
 - *X_train_XXX, X_test_XXX, y_train_XXX, y_test_XXX* transformed (XXX is the name of transformation) feature matrix, splitted into train and test sets
 

#### Normalization
For use in the machine-learning and especially deep learning frameworks the features have to be normalized:

In [ ]:
FeatureSet = FeatureSetDensePerc50

#from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn import decomposition
from sklearn.preprocessing import PolynomialFeatures

X, y = FeatureSet.drop(['CountyID', 'DiseaseR'], axis=1), FeatureSet['DiseaseR']
X_train0, X_test0, y_train0, y_test0 = train_test_split(X, y, random_state=0)


# default scaler: maps to [0:1]
scaler = MinMaxScaler()

scaler.fit(X_train0)
# apply transform
X_train = scaler.transform(X_train0)
X_test  = scaler.transform(X_test0)

# inverse transform
inverse = scaler.inverse_transform(X_train)
# also, maybe lognorm-like distribution can be converted to ~normal with box-cox.

checking inversibility of the scaling procedure:

In [ ]:
print(X_train0[0:4])
print(inverse[0:4])

#### Principal Component Analysis (PCA)
The 5 features from the initial dataset are projected onto 3 eigenvectors, corresponding to 3 extreme eigenvalues of the feature matrix:

In [ ]:
pca = decomposition.PCA(n_components=3)
pca.fit(X_train0)
print("PCA explained variance:", pca.explained_variance_ratio_)
X_train_PCA = pca.transform(X_train0)
X_test_PCA = pca.transform(X_test0)

#### Polynomial Features
In order to capture nonlinear behavior of the model,  one can extend the basis set from x<sub>1</sub>, x<sub>2</sub>, x<sub>3</sub> to 1, x<sub>1</sub>, x<sub>1</sub><sup>2</sup>, x<sub>2</sub>, x<sub>2</sub><sup>2</sup>, x<sub>3</sub>, x<sub>3</sub><sup>2</sup>:

In [ ]:
FeatureSet = FeatureSetLongPerc50
X, y = FeatureSet.drop(['CountyID', 'DiseaseR'], axis=1), FeatureSet['DiseaseR']
X_train0, X_test0, y_train0, y_test0 = train_test_split(X, y, random_state=0)


poly = PolynomialFeatures(2)
poly.fit_transform(X_train0)
X_train_UpToSquares = poly.transform(X_train0)
X_test_UpToSquares = poly.transform(X_test0)

In order to treat potential interactions between the feature variables,  one can extend the basis set from x<sub>1</sub>, x<sub>2</sub>, x<sub>3</sub> to 1, x<sub>1</sub>, x<sub>2</sub>, x<sub>3</sub>, x<sup>1</sup>x<sub>2</sub>, x<sub>1</sub>x<sub>3</sub>, x<sub>2</sub>x<sub>3</sub>, x<sub>1</sub>x<sub>2</sub>x<sub>3</sub>:

In [ ]:
poly = PolynomialFeatures(degree=3, interaction_only=True)
poly.fit_transform(X_train0)
X_train_UpToCube = poly.transform(X_train0)
X_test_UpToCube = poly.transform(X_test0)

#### Correlation matrix
The matrices with correlations between features can be plotted for both (Dense and Long) datasets:

In [ ]:
corr = FeatureSetDensePerc75.corr()
corr.style.background_gradient(cmap='coolwarm')

In [ ]:
corr = FeatureSetLongPerc75.corr()
corr.style.background_gradient(cmap='coolwarm')

It becomes clear, that the concentrations of different nitrogen oxides are highly correlated.
So, additional features engineering (e.g. polynomial) would be favorable w.r.t. keeping all the oxide's measurements.

### Additional feature creation, for classification approaches

Additional feature for the disease prevalence is constructed as presence of the county in Nth percentile of the disease prevalence:


In [ ]:
def DiseaseFeaturePercentile(FeatureSetDF, Percentile):
    DiseasePercentile = (FeatureSetDF['DiseaseR']).quantile(Percentile/100.0)
    dfFeatureOut = FeatureSetLongPerc75.copy(deep=True)
    dfFeatureOut['DiseaseR'] = ((dfFeatureOut['DiseaseR'])<DiseasePercentile)
    dfFeatureOut=dfFeatureOut.rename(columns = {'DiseaseR':'DiseaseRFeat'})
    return(dfFeatureOut)

dfPolMeanLongDisease50perc = DiseaseFeaturePercentile(FeatureSetLongMean, 50.0)
dfPolMeanLongDisease75perc = DiseaseFeaturePercentile(FeatureSetLongMean, 75.0)
dfPolMeanLongDisease95perc = DiseaseFeaturePercentile(FeatureSetLongMean, 95.0)

dfPolLongPerc75Disease50perc = DiseaseFeaturePercentile(FeatureSetLongPerc75, 50.0)
dfPolLongPerc75Disease75perc = DiseaseFeaturePercentile(FeatureSetLongPerc75, 75.0)
dfPolLongPerc75Disease95perc = DiseaseFeaturePercentile(FeatureSetLongPerc75, 95.0)

dfPolMeanLongDisease50perc.to_csv('Capstone.FeatureEng/Capstone.feature_eng.PolMeanLongDisease50perc.1.0.csv', index=False)
dfPolMeanLongDisease75perc.to_csv('Capstone.FeatureEng/Capstone.feature_eng.PolMeanLongDisease75perc.1.0.csv', index=False)
dfPolMeanLongDisease95perc.to_csv('Capstone.FeatureEng/Capstone.feature_eng.PolMeanLongDisease95perc.1.0.csv', index=False)

dfPolLongPerc75Disease50perc.to_csv('Capstone.FeatureEng/Capstone.feature_eng.PolLongPerc75Disease50perc.1.0.csv', index=False)
dfPolLongPerc75Disease75perc.to_csv('Capstone.FeatureEng/Capstone.feature_eng.PolLongPerc75Disease75perc.1.0.csv', index=False)
dfPolLongPerc75Disease95perc.to_csv('Capstone.FeatureEng/Capstone.feature_eng.PolLongPerc75Disease95perc.1.0.csv', index=False)
